<a href="https://colab.research.google.com/github/fabian-fuentes/DSGE/blob/main/HabitFormation/Lab_HabitFormation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Habit formation analysis


In [1]:
# install packages
install.packages('nleqslv')
install.packages('Rcpp')
install.packages('reshape2')
install.packages("https://gecon.r-forge.r-project.org/files/gEcon_1.2.3.tar.gz", repos = NULL, type = "source")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library(nleqslv)
library(Rcpp)
library(gEcon)

Loading required package: MASS

Loading required package: Matrix

This is gEcon version 1.2.3 (2025-04-13)
http://gecon.r-forge.r-project.org/



In [3]:
download.file("https://raw.githubusercontent.com/fabian-fuentes/DSGE/main/HabitFormation/hf_chapter3.gcn",
              destfile = "hf_chapter3.gcn", mode = "wb")

In [4]:
#create a model from gcc.file
model <- make_model("hf_chapter3.gcn")

model parsed in 0.02s
model loaded in 0.09s


In [5]:
#set parameter values

#set initial conditions
  model <- initval_var(model, list(K_s = 3.5,
                                   C = 0.8,
                                   L_s = 0.3 ,
                                   I = 0.2,
                                   Y = 1.0,
                                   A = 1.0,
                                   W=(1-0.350)*(1.0/0.3),
                                   r=(0.350)*(1.0/3.5),
                                   pi=1.0,
                                   PI=1.0   ))

Warning message in initval_var(model, list(K_s = 3.5, C = 0.8, L_s = 0.3, I = 0.2, :
“initial values of the following variables have not been set: "lambda__CONSUMER_2", "U"”


In [6]:
#define steady state
 model <- steady_state(model)

Warning message in ss_prep(model, use_jac, calibration, last_solver_iter):
“initial values for some variables have not been supplied (default value of 0.9 has been used).”


Steady state has been FOUND


In [7]:
#Now to explore output data
 get_ss_values(model)
 get_par_values(model)

Steady-state values:

                   Steady-state value
lambda__CONSUMER_2             0.7286
pi                             0.0000
r                              0.0909
A                              1.0000
C                              0.6148
I                              0.1846
K_s                            3.0774
L_s                            0.3869
PI                             0.0000
U                            -37.7293
W                              1.3431
Y                              0.7995
Model parameters:

      Value
alpha  0.35
beta   0.97
delta  0.06
gamma  0.40
phi    0.80
rho    0.95


In [8]:
#Solving for dynamics
 model <- solve_pert(model)

Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED


In [9]:
#printing the results
 get_pert_solution(model)


Matrix P:

       A[-1]   C[-1] K_s[-1]
A[]   0.9500  0.0000  0.0000
C[]   0.1771  0.7464  0.1040
K_s[] 0.2433 -0.0906  0.9481


Matrix Q:

    epsilon_A
A      1.0000
C      0.1864
K_s    0.2561


Matrix R:

                       A[-1]   C[-1] K_s[-1]
lambda__CONSUMER_2[] -0.7648  0.3402 -0.7103
pi[]                  0.0000  0.0000  0.0000
r[]                   1.0727  0.2254 -0.8887
I[]                   4.0549 -1.5095  0.1353
L_s[]                 0.1887  0.3468 -0.3673
PI[]                  0.0000  0.0000  0.0000
U[]                   0.1869 -0.0424  0.0613
W[]                   0.8839 -0.1214  0.4786
Y[]                   1.0727  0.2254  0.1113


Matrix S:

                   epsilon_A
lambda__CONSUMER_2   -0.8051
pi                    0.0000
r                     1.1291
I                     4.2683
L_s                   0.1987
PI                    0.0000
U                     0.1967
W                     0.9305
Y                     1.1291


In [10]:
#Introducing shocks
  model <- set_shock_distr_par(model = model, distr_par = list("sd(epsilon_A)" = 0.01))

In [11]:
#Print all results
  model<- compute_model_stats(model)
  get_model_stats(model)

Basic statistics:

                   Steady-state value Std. dev. Variance Loglin
lambda__CONSUMER_2             0.7286    0.0125    2e-04   Y   
pi                             0.0000    0.0000    0e+00   N   
r                              0.0909    0.0150    2e-04   Y   
A                              1.0000    0.0130    2e-04   Y   
C                              0.6148    0.0078    1e-04   Y   
I                              0.1846    0.0544    3e-03   Y   
K_s                            3.0774    0.0106    1e-04   Y   
L_s                            0.3869    0.0026    0e+00   Y   
PI                             0.0000    0.0000    0e+00   N   
U                            -37.7293    0.0027    0e+00   Y   
W                              1.3431    0.0137    2e-04   Y   
Y                              0.7995    0.0156    2e-04   Y   

Correlation matrix:

                   lambda__CONSUMER_2      r      A      C      I    K_s    L_s
lambda__CONSUMER_2                  1 -0.616 -0

In [12]:
#Compute Impulse Response Function (IRFs)
  model_irf <- compute_irf(model = model, variables = c('C', 'K_s', 'A', 'Y', 'L_s', 'I'), sim_length = 40)

In [13]:
#Extract simulation values
library(reshape2)

  out <- as.data.frame.table(model_irf@sim)
  out<-dcast(out,Var2  ~ Var1 , value.var="Freq")
  out$Case<-"habit formation"

In [14]:
#run another case
  model1 <- set_free_par(model, list(phi = 0.0))
  model1 <- initval_var(model1, list(K_s = 3.5, C = 0.8, L_s = 0.3 , I = 0.2, Y = 1.0, A = 0.5))
  model1 <- steady_state(model1)
  model1 <- set_shock_distr_par(model = model1, distr_par = list("sd(epsilon_A)" = 0.01))
  model1 <- solve_pert(model1)
  model_irf1 <- compute_irf(model = model1, variables = c('C', 'K_s', 'A', 'Y', 'L_s', 'I'), sim_length = 40)
  out1 <- as.data.frame.table(model_irf1@sim)
  out1<-dcast(out1,Var2  ~ Var1 , value.var="Freq")
  out1$Case<-"Cannonical Model"

Warning message in set_free_par(model, list(phi = 0)):
“the values of the following parameters have been overwritten: "phi"”
Warning message in initval_var(model1, list(K_s = 3.5, C = 0.8, L_s = 0.3, I = 0.2, :
“initial values of the following variables have not been set: "lambda__CONSUMER_2", "pi", "r", "PI", "U", "W"”


Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED


In [15]:
#rbind
  out<-rbind(out,out1)

In [16]:
#print simulation
  write.csv(out,"out.csv",row.names=FALSE)

In [17]:
#Now let's compare how the intensity of the productivy shock affects the model
sdepsilon_Avector<-seq(0.5,1.5, by= 0.1)
out_all <- list()

In [18]:
for (i in 1:length(sdepsilon_Avector))
{
#i<-2
#create a model from gcc.file
  model <- make_model("hf_chapter3.gcn")
#set initial conditions
  model <- initval_var(model, list(K_s = 3.5,
                                   C = 0.8,
                                   L_s = 0.3 ,
                                   I = 0.2,
                                   Y = 1.0,
                                   A = 1.0,
                                   W=(1-0.350)*(1.0/0.3),
                                   r=(0.350)*(1.0/3.5),
                                   pi=1.0,
                                   PI=1.0   ))
#define steady state
  model <- steady_state(model)
#Solving for dynamics
 model <- solve_pert(model)
# Introduce Shock
 model <- set_shock_distr_par(model = model, distr_par = list("sd(epsilon_A)" = sdepsilon_Avector[i]*0.01))
#Compute Impulse Response Function (IRFs)
  model_irf <- compute_irf(model = model, variables = c('C', 'K_s', 'A', 'Y', 'L_s', 'I'), sim_length = 40)
#Extract simulation values
  out <- as.data.frame.table(model_irf@sim)
  out<-dcast(out,Var2  ~ Var1 , value.var="Freq")
  out$Case<-paste0("sd_epsilon_A = ",sdepsilon_Avector[i])
#save ouput
 out_all <- append(out_all,list(out))
}

model parsed in 0.01s
model loaded in 0.02s


Warning message in initval_var(model, list(K_s = 3.5, C = 0.8, L_s = 0.3, I = 0.2, :
“initial values of the following variables have not been set: "lambda__CONSUMER_2", "U"”
Warning message in ss_prep(model, use_jac, calibration, last_solver_iter):
“initial values for some variables have not been supplied (default value of 0.9 has been used).”


Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s


...

Model has been SOLVED
model parsed in 0.01s
model loaded in 0.01s


Warning message in initval_var(model, list(K_s = 3.5, C = 0.8, L_s = 0.3, I = 0.2, :
“initial values of the following variables have not been set: "lambda__CONSUMER_2", "U"”
Warning message in ss_prep(model, use_jac, calibration, last_solver_iter):
“initial values for some variables have not been supplied (default value of 0.9 has been used).”


Steady state has been FOUND


Warning message:
“the following variables will not be log-linearised as their steady-state values are equal or close to zero: "pi", "PI"”


Model has been SOLVED


In [19]:
out_all <- do.call("rbind",out_all)
head(out_all)

,Var2,C,K_s,A,Y,L_s,I,Case
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,0.0009321074,0.001280487,0.005000000,0.005645696,0.0009933778,0.02134146,sd_epsilon_A = 0.5
2,2,0.0017144603,0.002346094,0.004750000,0.005715972,0.0007966168,0.01904059,sd_epsilon_A = 0.5
3,3,0.0023650086,0.003224732,0.004512500,0.005742700,0.0006293339,0.01699006,sd_epsilon_A = 0.5
4,4,0.0028999377,0.003941081,0.004286875,0.005732326,0.0004873771,0.01516389,sd_epsilon_A = 0.5
5,5,0.0033337860,0.004516923,0.004072531,0.005690574,0.0003671766,0.01353844,sd_epsilon_A = 0.5
6,6,0.0036795656,0.004971446,0.003868905,0.005622508,0.0002656616,0.01209230,sd_epsilon_A = 0.5


In [20]:
#print simulation
 write.csv(out_all,"out_sentivity.csv",row.names=FALSE)